# Customer Churn Prediction with Autogluon using Script mode
_**이동통신 고객 이탈감지를 위해 Gradient Boosted Trees 사용하기**_

소스 : https://github.com/aws/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Compile](#Compile)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

---

## Background

_본 노트북의 내용은 다음 블로그에서도 확인할 수 있습니다. [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

어떤 비즈니스이건 고객을 잃는 것은 손해로 연결됩니다. 불만족스러운 고객을 조기에 감지할 수 있다면 여러분은 그들에게 인센티브를 제공함으로써 보다 더 오래 머물도록 할 기회를 줄 것입니다. 본 노트북은 고객 이탈 예측(Churn Prediction)으로 잘 알려진, 불만족스러운 고객을 사전에 식별하는 유즈케이스를 머신러닝을 이용하여 자동으로 해결하고자 합니다. 머신러닝 모델은 완벽하게 예측하지는 못할 것입니다. 본 노트북은 이런 경우 어떻게 예측이 빗나간 케이스에 대하여 관련된 비용을 수용하는지까지 다룰 것입니다.

고객 이탈 예측 예제에서 우리에게 익숙한 이동통신사 예를 사용할 것입니다. 서비스제공자는 고객이 탈퇴할 생각임을 알게 되면 적절한 타이밍에 인센티브를 제공할 수 있습니다. 전화기를 업그레이드하거나 새로운 기능을 활성화하여 계속 서비스를 사용하도록 합니다. 인센티브는 고객을 잃고 다시 확보하는 것보다 훨씬 비용 효율적인 경우가 많습니다. 


---

## Setup

_본 노트북은 ml.m4.xlarge Sagemaker 노트북에서 생성하고 테스트되었습니다._

다음 설정을 시작합니다.
- 학습과 모델 데이터 저장에 사용할 S3 버켓과 prefix를 선언합니다. SageMaker 노트북 인스턴스와 동일한 리전에 위치해야 합니다.
- 학습과 호스팅 작업에서 데이터에 엑서스할 때 사용할 IAM 역할(role)을 결정합니다. 역할의 생성은 aws 개발자문서를 참고하십시오. 만약 노트북인스턴스, 학습, 호스팅을 위해 하나 여러개의 별도 역할이 필요하다면 boto 정규식으로 표현된 IAM 풀네임 스트링을 사용합니다. 


In [1]:
import sagemaker
import pandas as pd
import numpy as np
from utils.ag_model import (
    AutoGluonTraining,
    AutoGluonInferenceModel,
    AutoGluonTabularPredictor,
)
from sagemaker import utils
from sagemaker.serializers import CSVSerializer
import os

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name

bucket = sagemaker_session.default_bucket()
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

필요한 파이썬 라이브러리를 import 합니다. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

---
## Data

이동 통신사에는 어떤 고객이 이탈하고 어떤 고객이 서비스를 계속 사용했는지에 대한 기록이 있습니다. 이 기록 정보를 이용하여 한 이동 통신사의 이탈에 대한 ML 모델을 구성하기 위해 모델을 학습할  할 수 있습니다. 모델을 학습 한 후 다른 임의 고객의 프로필 정보 (모델 학습에 사용한 것과 동일한 프로필 정보)를 모델에 전달하고 모델이이 고객이 이탈할지 여부를 예측하도록 할 수 있습니다. 물론 우리는 모델이 실수를 할 것으로 예상합니다. 결국 미래를 예측하는 것은 어려운 일입니다! 하지만 예측 오류를 처리하는 방법도 보여 드리겠습니다.

우리가 사용하는 데이터 세트는 Daniel T. Larose의 책 [Discovering Knowledge in Data] (https://www.amazon.com/dp/0470908742/) 과 University of California Irvine Repository of Machine Learning Datasets에 언급되고 공개되었습니다. 이제 해당 데이터 세트를 다운로드하고 읽어 보겠습니다.

In [3]:
%store -r train_data validation_data test_data predictions

In [4]:
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)

label = 'Churn?_True.'
print("Summary of |class variable: \n", train_data[label].describe())

y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating

Summary of |class variable: 
 count    500.000000
mean       0.510000
std        0.500401
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Churn?_True., dtype: float64


In [5]:
train_data.to_csv("train_header.csv", header=True, index=False)
validation_data.to_csv("validation_header.csv", header=True, index=False)

---
## Train

Users can create their own training/inference scripts using [SageMaker Python SDK examples](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script).
The scripts we created allow to pass AutoGluon configuration as a YAML file (located in `data/config` directory).

We are using [official AutoGluon Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers) with custom training scripts (see `scripts/` directory).

In [6]:
ag = AutoGluonTraining(
    role=role,
    entry_point="scripts/tabular_train.py",
    region=region,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    framework_version="0.3.1",
    base_job_name="autogluon-tabular-train",
)

In [7]:
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
train_input = ag.sagemaker_session.upload_data(
    path="train_header.csv", key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path="validation_header.csv", key_prefix=s3_prefix
)
config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-med.yaml"), key_prefix=s3_prefix
)

### Fit The Model
For local training set `instance_type` to local.

For non-local training the recommended instance type is `ml.m5.2xlarge`.

In [8]:
job_name = utils.unique_name_from_base("test-autogluon-image")
ag.fit(
    {"config": config_input, "train": train_input, "test": eval_input},
    job_name=job_name,
)

2021-12-05 14:05:44 Starting - Starting the training job...
2021-12-05 14:06:07 Starting - Launching requested ML instancesProfilerReport-1638713143: InProgress
......
2021-12-05 14:07:07 Starting - Preparing the instances for training......
2021-12-05 14:08:14 Downloading - Downloading input data...
2021-12-05 14:08:27 Training - Downloading the training image..2021-12-05 14:08:56,791 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-12-05 14:08:56,793 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-05 14:08:56,803 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'tabular_train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{"config":{"RecordWrapperType":"None","

### 모델 및 결과 추출

AutoGluon models are portable: everything needed to deploy a trained model is in the tarball created by SageMaker.

The artifact can be used locally, on EC2/ECS/EKS or served via SageMaker Inference.

In [9]:
!aws s3 cp {ag.model_data} .

download: s3://sagemaker-us-west-2-322537213286/test-autogluon-image-1638713143-b199/output/model.tar.gz to ./model.tar.gz


In [10]:
!ls -alF model.tar.gz

-rw-rw-r-- 1 ec2-user ec2-user 7521020 Dec  5 14:09 model.tar.gz


In [11]:
artifacts_dir = ag.model_data.replace('model.tar.gz', '')
output_dir = "./train_output"

In [12]:
!rm -rf {output_dir}
!mkdir {output_dir}
!aws s3 cp {artifacts_dir}output.tar.gz {output_dir}/output.tar.gz
!tar -xzf {output_dir}/output.tar.gz -C {output_dir}

download: s3://sagemaker-us-west-2-322537213286/test-autogluon-image-1638713143-b199/output/output.tar.gz to train_output/output.tar.gz


In [13]:
pd.read_csv(f'{output_dir}/leaderboard.csv')

,Unnamed: 0,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,0,CatBoost_BAG_L1,0.968310,0.940376,0.015566,0.014677,0.886339,0.015566,0.014677,0.886339,1,True,7
1,1,WeightedEnsemble_L2,0.966229,0.947371,0.640336,0.622511,6.688812,0.004611,0.001395,1.603110,2,True,14
2,2,LightGBMLarge_BAG_L1,0.963691,0.938023,0.015885,0.010091,0.666660,0.015885,0.010091,0.666660,1,True,13
3,3,RandomForestGini_BAG_L1,0.962877,0.933421,0.112377,0.112579,0.699357,0.112377,0.112579,0.699357,1,True,5
4,4,RandomForestEntr_BAG_L1,0.961554,0.935454,0.115260,0.121136,0.664288,0.115260,0.121136,0.664288,1,True,6
5,5,LightGBM_BAG_L1,0.959245,0.929716,0.011093,0.007519,0.327012,0.011093,0.007519,0.327012,1,True,4
6,6,XGBoost_BAG_L1,0.957560,0.931236,0.031993,0.019107,0.401591,0.031993,0.019107,0.401591,1,True,11
7,7,LightGBMXT_BAG_L1,0.954986,0.927299,0.015156,0.007953,1.167132,0.015156,0.007953,1.167132,1,True,3
8,8,NeuralNetMXNet_BAG_L1,0.954454,0.916158,0.266068,0.074241,6.577654,0.266068,0.074241,6.577654,1,True,12
9,9,ExtraTreesEntr_BAG_L1,0.943912,0.924730,0.113843,0.116916,0.768242,0.113843,0.116916,0.768242,1,True,9


# Endpoint Deployment

Upload the model we trained earlier

In [14]:
endpoint_name = sagemaker.utils.unique_name_from_base("sagemaker-autogluon-serving-trained-model")

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [15]:
instance_type = "ml.m5.2xlarge"
# instance_type = 'local'

In [16]:
model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.3.1",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="tabular_serve.py",
)

In [17]:
predictor = model.deploy(
    initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type
)

-----!

### Predict on unlabeled test data

Remove target variable (`class`) from the data and get predictions for a sample of 100 rows using the deployed endpoint.

In [18]:
test_data.reset_index(drop=True, inplace=True)

In [19]:
data = test_data.drop(columns="Churn?_True.")[:100].values

In [20]:
preds = predictor.predict(data)

In [21]:
pd.crosstab(
    index=test_data["Churn?_True."].astype("float")[: len(preds)],
    columns=pd.DataFrame(preds)[0],
    rownames=["actual"],
    colnames=["preds"],
)

preds,0.0,1.0
actual,,
0.0,48,5
1.0,3,44


### Cleanup Endpoint

In [22]:
predictor.delete_endpoint()

# Batch Transform

Deploying a trained model to a hosted endpoint has been available in SageMaker since launch and is a great way to provide real-time predictions to a service like a website or mobile app. But, if the goal is to generate predictions from a trained model on a large dataset where minimizing latency isn’t a concern, then the batch transform functionality may be easier, more scalable, and more appropriate.

[Read more about Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).

In [23]:
endpoint_name = sagemaker.utils.unique_name_from_base(
    "sagemaker-autogluon-batch_transform-trained-model"
)

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [24]:
instance_type = "ml.m5.2xlarge"

In [25]:
model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.3.1",
    instance_type=instance_type,
    entry_point="tabular_serve-batch.py",
    source_dir="scripts",
    predictor_cls=AutoGluonTabularPredictor,
)

In [26]:
transformer = model.transformer(
    instance_count=1,
    instance_type=instance_type,
    strategy="MultiRecord",
    max_payload=6,
    max_concurrent_transforms=1,
    output_path=output_path,
    accept="application/json",
    assemble_with="Line",
)

Prepare data for batch transform

In [27]:
test_data[:100].to_csv("test_no_header.csv", header=False, index=False)

Upload data to sagemaker session

In [28]:
test_input = transformer.sagemaker_session.upload_data(
    path="test_no_header.csv", key_prefix=s3_prefix
)

In [29]:
transformer.transform(
    test_input,
    input_filter="$[1:]",  # filter-out target variable
    split_type="Line",
    content_type="text/csv",
#     output_filter="$[0]",  # keep only prediction class in the output
)

transformer.wait()

2021-12-05 14:18:13,706 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /usr/local/lib/python3.7/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 8
Max heap size: 6064 M
Python executable: /usr/local/bin/python3.7
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 8
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Preload model: false
Prefer direct buffer: false
2021-12-05 14:18:13,777 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-9000-model
2021-12-05 14:18:13,855 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - model_service_worker started with args: --sock-type unix --sock-name /home/mod

Download batch transform outputs

In [30]:
!aws s3 cp {transformer.output_path[:-1]}/test_no_header.csv.out .

download: s3://sagemaker-us-west-2-322537213286/autogluon_sm/2021-12-05-14-05-42-648/output/test_no_header.csv.out to ./test_no_header.csv.out


In [31]:
result = pd.read_json("test_no_header.csv.out", orient="index").sort_index()[1:].T
result.columns = ["preds", "actual"]
result['preds'] = result['preds'].apply(lambda x: 1.0 if x>=0.5 else 0.0)

In [36]:
pd.crosstab(
    index=result["actual"],
    columns=result["preds"],
    rownames=["actual"],
    colnames=["preds"],
)

preds,0.0,1.0
actual,,
0.0,51,0
1.0,0,49


# Conclusion

In this tutorial we successfully trained an AutoGluon model and explored a few options how to deploy it using SageMaker. Any of the sections of this tutorial (training/endpoint inference/batch inference) can be used independently (i.e. train locally, deploy to SageMaker, or vice versa).

Next steps:
* [Learn more](https://auto.gluon.ai) about AutoGluon, explore [tutorials](https://auto.gluon.ai/stable/tutorials/index.html).
* Explore [SageMaker inference documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).